	
# Types produits
## Produits cartésiens
On peut, si on le souhaite, définir un type correspondant à un produit cartésien :

In [1]:
type complexe = float * float;;

type complexe = float * float


Cela pourra être utile lorsque nous définirons d'autres types (voir ci-dessous), mais OCaml n'utilisera pas ce type (synonyme de `float*float` ) par défaut :

In [2]:
let module_complexe z =
  let  a, b = z in sqrt(a**2. +. b**2.);;

val module_complexe : float * float -> float = <fun>


## Enregistrements
Lorsqu'on souhaite représenter une entité décrite par un ensemble d'informations, par exemple représenter un étudiant par son nom, son prénom et sa date de naissance, on peut utiliser un $n$-uplet, dans notre exemple un quintuplet constitué de deux chaînes de caractères (nom et prénom) et de trois entiers (année, mois et jour de naissance).

Manipuler les informations liées à ces objets est alors assez délicat : il faut gérer l'ordre dans lequel les informations sont données.

Il existe une autre catégorie de types produits en OCaml : les enregistrements. Alors que dans un $n$-uplets, les différentes informations ne sont repérées que par leur position, dans un enregistrement, chaque composante possède un nom.

Il est possible de définir un nouveau type enregistrement, par exemple un type `date`  contenant trois *champs* (ses composantes) de type `int`  : un jour, un mois et une année.

In [3]:
type date = {
    jour : int;
    mois : int;
    annee : int;
  };;

type date = { jour : int; mois : int; annee : int; }


Pour construire une date, par exemple le 14 mars 2022 :

In [4]:
let d = {annee = 2022; mois = 3; jour = 14;};;

val d : date = {jour = 14; mois = 3; annee = 2022}


On remarque qu'il n'est pas nécessaire de donner les champs dans le même ordre que dans la définition.

Ce type `date`  peut être utilisé pour construire le type `etudiant`  :

In [5]:
type etudiant = {
    nom : string;
    prenom : string;
    naissance : date;
  };;

type etudiant = { nom : string; prenom : string; naissance : date; }


In [6]:
let e = {
    nom = "Tournesol";
    prenom = "Tryphon";
    naissance = {
        jour = 28 ;
        mois = 1;
        annee = 2003;
      }
  };;

val e : etudiant =
  {nom = "Tournesol"; prenom = "Tryphon";
   naissance = {jour = 28; mois = 1; annee = 2003}}


Pour accéder à l'un des champs d'un objet de type enregistrement, il suffit de faire suivre le nom de l'objet d'un point puis du nom du champ.

In [7]:
e.naissance;;

- : date = {jour = 28; mois = 1; annee = 2003}


In [8]:
let est_majeur e date =
  let n = e.naissance in
  let j = n.jour and m = n.mois and a = n.annee in
  let age = date.annee - a - (if m > date.mois || (m = date.mois && j > date.jour) then 1 else 0) in
  age >= 18
;;

val est_majeur : etudiant -> date -> bool = <fun>


In [9]:
let aujourdhui = {jour = 8 ; mois = 2; annee = 2022}

val aujourdhui : date = {jour = 8; mois = 2; annee = 2022}


In [10]:
est_majeur e aujourdhui;;

- : bool = true


# Types sommes
## Types énumérations
Jusqu'à présent, toutes les données que nous pouvons représenter reposent sur les types standards. Comment représenter alors les jours de la semaine ?

- Par un entier : quelle convention choisir ? Numérote-t-on de $0$ à $6$ ou de $1$ à $7$ ? En commençant par quel jour ? 
- Ou par une chaîne de caractères.

Dans les deux cas, si on utilise une donnée qui ne correspond à aucun jour (par exemple en faisant une faute de frappe dans une chaîne de caractères), cela ne sera détecté qu'à l'exécution.

On peut alors utiliser un type *énumération*, en donnant toutes les valeurs possibles :

In [11]:
type jour_semaine =
  | Lundi
  | Mardi
  | Mercredi
  | Jeudi
  | Vendredi
  | Samedi
  | Dimanche
;;

type jour_semaine =
    Lundi
  | Mardi
  | Mercredi
  | Jeudi
  | Vendredi
  | Samedi
  | Dimanche


Les valeurs `Lundi` ,... sont appelées les *constructeurs* du type `jour_semaine` . Ce sont en effet les seules opérations permettant de construire des objets du type `jour_semaine` . Les constructeurs doivent commencer par une majuscule.

In [12]:
Lundi;;

- : jour_semaine = Lundi


In [13]:
Mardi;;

- : jour_semaine = Mardi


On peut réaliser un filtrage sur les valeurs du type `jour_semaine` 

In [15]:
let week_end j =
  match j with
  | Samedi -> true
  | Dimanche -> true
  | _ -> false
;;

val week_end : jour_semaine -> bool = <fun>


In [16]:
week_end Mardi;;

- : bool = false


Utiliser un type énumération demande néanmoins que le nombre de valeurs possibles soit fini, comme pour les jours de la semaine ou les couleurs d'une carte à la belote.

In [17]:
type couleur =
  | Trefle
  | Carreau
  | Coeur
  | Pique
;;

type couleur = Trefle | Carreau | Coeur | Pique


## Types sommes
Il s'agit d'une généralisation des types énumérations. Supposons que nous souhaitions représenter les cartes d'un jeu de 32 cartes. Chaque carte est alors identifiée par sa couleur (Trèfle, Carreau, Cœur, Pique) et sa valeur (As, Roi, Dame, valet, 10, 9, 8, 7).

In [18]:
type carte =
  | As of couleur
  | Roi of couleur
  | Dame of couleur
  | Valet of couleur
  | Petite_carte of int * couleur
;;

type carte =
    As of couleur
  | Roi of couleur
  | Dame of couleur
  | Valet of couleur
  | Petite_carte of int * couleur


Comme précédemment, `As` , `Petite_carte`  sont des constructeurs :

In [21]:
let ma_carte = Valet Trefle;;

val ma_carte : carte = Valet Trefle


In [22]:
Petite_carte (9, Pique);;

- : carte = Petite_carte (9, Pique)


Il est possible d'effectuer un filtrage sur ces valeurs :

In [23]:
let valeur_carte atout = fun carte ->
  match carte with
  | As _ -> 11
  | Roi _ -> 4
  | Dame _ -> 3
  | Valet c -> if c = atout then 20 else 2
  | Petite_carte (10, _) -> 10
  | Petite_carte (9, c) -> if c = atout then 14 else 0
  | _ -> 0
;;

val valeur_carte : couleur -> carte -> int = <fun>


In [24]:
let toto = Valet Coeur in
valeur_carte Coeur toto;;

- : int = 20


Un type somme peut être *récursif*, c'est à dire intervenir dans sa propre définition.
Définissons par exemple les couleurs par synthèse soustractive, en considérant qu'une couleur est soit une couleur primaire, soit un mélange de deux couleurs :

In [25]:
type color =
  | Cyan
  | Magenta
  | Jaune
  | Melange of color * color;;

type color = Cyan | Magenta | Jaune | Melange of color * color


In [26]:
let rouge = Melange (Magenta, Jaune);;

val rouge : color = Melange (Magenta, Jaune)


In [27]:
let orange = Melange (rouge, Jaune);;

val orange : color = Melange (Melange (Magenta, Jaune), Jaune)


# Exercices divers

## Exercice 1
- Définir un type enregistrement `complexe`  associé aux nombres complexes.

- Écrire une fonction `module : complexe -> float`  calculant le module d'un nombre complexe.

- Écrire une fonction `produit : complexe -> complexe -> complexe`  calculant le produit de deux nombres complexes.

## Exercice 2
- Définir un type somme `reel_etendu`  permettant de représenter la droite numérique achevée.

- Écrire une fonction `etendu_of_float`  permettant de convertir un nombre de type `float`  en un nombre de type `reel_etendu` .

- Écrire une fonction `somme : reel_etendu -> reel_etendu -> reel_etendu`  calculant (si c'est possible) la somme de deux réels étendus.